In [1]:
import os
import glob
import pandas as pd
from bs4 import BeautifulSoup

### 1) Varrer os textos do Arquivo Pessoa

Na linha de comandos, executar:

``` for i in {0..9999}; do wget http://arquivopessoa.net/textos/$i; done ```

### 2) Iterar os ficheiros HTML

In [2]:
#ir buscar os ficheiros html
file_location = os.path.join('pessoa-textos', '*.html')

#initailizing empty list; each html will be converted to a dictionary
dict_lst = []

for file in glob.glob(file_location):
    with open(file) as fp:
        
        soup = BeautifulSoup(fp)
        
        #get author
        author = soup.find("div", attrs = {"class": "autor"}).string
            
        #get title + title cleaning
        text_title = soup.find("h1", attrs = {"class": "titulo-texto"})
        clean_title = text_title.text.replace('\n', '').replace('\xa0', '')  
            
        #get date
        date = soup.find("div", attrs = {"class": "data"}).string 
            
        try:
            #get full poem + poem tidying
            full_text = soup.find("div", attrs = {"class": "texto-poesia"})
            clean_text = full_text.text.replace('<p>', '').replace('\xa0', '').replace('</p>','\n')
            poem_text_dict = {}
            for variable in ["author", "clean_title", "clean_text", "date"]:
                poem_text_dict[variable] = eval(variable)
            poem_text_dict["text_type"] = "poem"
            dict_lst.append(poem_text_dict)
        
        except:
            #get full poem + prose tidying
            full_text = soup.find("div", attrs = {"class": "texto-prosa"})
            clean_text = full_text.text.replace('<p>', '').replace('\xa0', '').replace('</p>','\n')       
            prose_text_dict = {}
            for variable in ["author", "clean_title", "clean_text", "date"]:
                prose_text_dict[variable] = eval(variable)
            prose_text_dict["text_type"] = "prose"
            dict_lst.append(prose_text_dict)
            
    full_data = pd.DataFrame(dict_lst)

In [3]:
#adjusting column names and exporting to csv
full_data.rename(columns={"clean_title":"title", "clean_text":"text"}, inplace=True)
full_data.to_csv("pessoa_EN.csv")

In [4]:
#adjusting stuff to portuguese and exporting to csv
full_data_PT = full_data
full_data_PT.rename(columns={"author":"autor", "title":"título","text":"texto","date":"data", "text_type":"tipo"}, inplace=True)
full_data_PT.replace({'tipo': {"poem": "poema", "prose": "prosa"}}, inplace=True)
full_data.to_csv("pessoa_PT.csv")
